In [ ]:
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv
from pathlib import Path
from research_tools.gpu import get_gpus_available

load_dotenv()


hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

n_gpus = 1

gpus_available = get_gpus_available()
n_gpus = min(n_gpus, len(gpus_available))
gpus = gpus_available[:n_gpus]

assert n_gpus > 0, "No GPUs available"

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in gpus])

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List, Dict, Optional
import torch
from research_tools.utils import set_seed

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "HuggingFaceH4/zephyr-7b-beta"

model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
from relearn.datasets.utils import (
    load_dataset as local_load_dataset,
    DATASETS_DICT,
    Datasets,
)
from relearn.datasets.corpus import process as process_corpus
from relearn.datasets.mcq import process as process_mcq
import pickle

data_dir = Path("../data")
cache_path = data_dir / "full.pickle"

USE_CACHE = True

if USE_CACHE:
    assert cache_path.exists(), "Cache file does not exist"
    with open(cache_path, "rb") as f:
        data = pickle.load(f)
else:
    data = {}
    # iterate over all enums
    for name in Datasets:

        dataset_config = DATASETS_DICT[name]

        def get_dataset(train_files: List[str], val_files: List[str], max_length: int):
            train = local_load_dataset(data_dir, train_files)
            val = local_load_dataset(data_dir, val_files)
            train_records = process_corpus(train, tokenizer, max_length)
            val_records = process_mcq(val, tokenizer, max_length)
            mcq_records = process_mcq(val, tokenizer, max_length, expand_choices=False)
            return {
                "corpus": train_records,
                "mcq": mcq_records,
                "val": val_records,
            }

        max_length = 512

        unlearn_files = dataset_config["unlearn_files"]
        val_unlearn_files = dataset_config["val_unlearn_files"]

        print(f"Processing {name}")

        data[name] = get_dataset(unlearn_files, val_unlearn_files, max_length)

        if "retain" not in data:
            retain_files = dataset_config["retain_files"]
            val_retain_files = dataset_config["val_retain_files"]
            data["retain"] = get_dataset(retain_files, val_retain_files, max_length)

    with open(cache_path, "wb") as f:
        pickle.dump(data, f)

In [4]:
import torch


def group_shuffle(data: List, group_size: int = 1, perm: Optional[torch.Tensor] = None):
    if perm is None:
        n = len(data) // group_size
        perm = torch.randperm(n)

    res = []
    for i in perm:
        res += data[i * group_size : (i + 1) * group_size]
    return res


def create_k_folds(data: List, k: int, group_size: int = 1):
    n = len(data) // group_size
    fold_size = n // k
    folds = [fold_size] * k
    for i in range(n % k):
        folds[i] += 1

    assert sum(folds) == n

    res = []
    start = 0

    for fold_size in folds:
        start_idx = start * group_size
        end_idx = (start + fold_size) * group_size

        res.append(data[start_idx:end_idx])

        start += fold_size

    return res


def get_folds_shuffled(records: Dict[str, List], k: int):

    perm = torch.randperm(len(records["mcq"]))

    store = [
        {"corpus": c, "mcq": m, "val": v}
        for c, m, v in zip(
            create_k_folds(group_shuffle(records["corpus"], 3, perm=perm), k, 3),
            create_k_folds(group_shuffle(records["mcq"], perm=perm), k),
            create_k_folds(
                group_shuffle(
                    records["val"],
                    4,
                    perm=perm,
                ),
                k,
                4,
            ),
        )
    ]

    return store


records = data[Datasets.WMDP]
k = 3

store = get_folds_shuffled(records, k)

In [5]:
from relearn.unlearn.rmu import train_rmu
import itertools

import wandb


def super_rmu(
    model: AutoModelForCausalLM,
    tokenizer: AutoTokenizer,
    forget_records_dict: Dict[str, Dict],
    retain_records_dict: Dict[str, Dict],
    eval_records_dict: Dict[str, Dict],
    k_folds: int,
    lr: float = 1e-5,
    joint_train: bool = False,
    prefix_forget: bool = True,
):
    assert k_folds <= 26, "k_folds must be less than 26"

    def fold_name(i: int):
        return chr(ord("A") + i)

    folds = get_folds_shuffled(forget_records_dict, k_folds)

    def get_data(fold_inds: List[int]):
        if joint_train:
            return {fold_name(i): folds[i]["corpus"] for i in fold_inds}
        else:
            return {
                fold_name(-1): list(
                    itertools.chain(
                        *[f[k]["corpus"] for i, f in enumerate(folds) if i in fold_inds]
                    )
                )
            }

    if eval_records_dict is None:
        eval_records_dict = {fold_name(i): folds[i]["val"] for i in range(k_folds)}
        eval_records_dict["retain"] = retain_records_dict["val"]

    # intuition: forget alpha decreases
    # intuition: retain alpha increases
    # priors
    # sweep across schedules, and epochs????
    # learning rate should probably go down?

    # keep retain retain alpha constant
    # maybe not do prefix, maybe the bumbling around is what's good?
    # try everything
    base_epoch = 0
    n_epochs = 2
    control_vecs = {}

    for i in range(k_folds):
        print(f"Unlearning fold {fold_name(i)}")

        if prefix_forget:
            forget_fold_inds = list(range(i + 1))
        else:
            forget_fold_inds = [i]

        retain_fold_inds = list(range(i + 1, k_folds))

        forget_dict = get_data(forget_fold_inds)
        retain_dict = get_data(retain_fold_inds)
        retain_dict["retain"] = retain_records_dict["corpus"]

        # weird retain coef cause i finetuned for 2/2
        model, control_vecs_next = train_rmu(
            model,
            forget_dict,
            retain_dict,
            eval_records_dict,
            magnitude=6.5,
            forget_alphas={k: 0.39422 * 2 / (i + 1 + 1e-6) for k in forget_dict.keys()},
            retain_alphas={
                **{
                    k: 13.51609 * 2 / (k_folds - i - 1 + 1e-6)
                    for k in retain_dict.keys()
                },
                **{
                    "retain": 1.0,
                },
            },
            lr=lr,
            tokenizer=tokenizer,
            use_wandb=True,
            eval_at_start=True if i == 0 else False,
            n_epochs=n_epochs,
            max_batches=None,
            base_epoch=base_epoch,
            return_control_vecs=True,
            control_vecs_init=control_vecs,
            print_evals=True,
        )
        control_vecs.update(control_vecs_next)
        base_epoch += n_epochs

    return model


wandb.login()

config = {
    "model_id": model_id,
    "magnitude": 6.5,
    "lr": 1e-5,
    "n_epochs": 12,
    "forget_alphas": {"A": 0.39422},
    "retain_alphas": {"B": 13.51609, "retain": 1},
}


run = wandb.init(
    project="relearn", config=config, tags=["rmu", "fold", "debug"], entity="12tqian"
)

model = super_rmu(
    model,
    tokenizer,
    data[Datasets.WMDP],
    data["retain"],
    None,
    4,
    lr=config["lr"],
    joint_train=True,
)

run.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 12tqian. Use `wandb login --relogin` to force relogin


Unlearning fold A


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:42<00:00,  9.34it/s]


{'epoch': -1, 'A/acc': 0.5228426395939086, 'B/acc': 0.49489795918367346, 'C/acc': 0.576530612244898, 'D/acc': 0.5969387755102041, 'retain/acc': 0.5923566878980892}


  0%|                                                                                                                                                                                                              | 0/147 [00:00<?, ?it/s]/mnt/align4_drive/tcqian/unlearn_order/src/relearn/unlearn/rmu/unlearn.py:242: UserWarning: Using a target size (torch.Size([1, 1, 4096])) that is different to the input size (torch.Size([4, 512, 4096])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  forget_loss = torch.nn.functional.mse_loss(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:42<00:00,  9.31it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:41<00:00,  9.38it/s]


{'epoch': 6, 'A/acc': 0.1319796954314721, 'B/acc': 0.18877551020408162, 'C/acc': 0.25510204081632654, 'D/acc': 0.22959183673469388, 'retain/acc': 0.4547770700636943}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 393/393 [00:42<00:00,  9.29it/s]

{'epoch': 7, 'A/acc': 0.13705583756345177, 'B/acc': 0.1989795918367347, 'C/acc': 0.22448979591836735, 'D/acc': 0.22959183673469388, 'retain/acc': 0.42420382165605097}


A/acc,██▇▅▄▃▂▁▁
A/forget_loss,█████████████▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁
B/acc,████▇▇▆▁▁
B/forget_loss,▆▆▆▆▆▆▆▆▆▆▆█▆▆▆▃▄▃▃▃▅▃▃▃▃▂▃▃▁▁▁▂▁▁▁▁▁▁▁▁
B/retain_loss,▁▂▂▂▂▁▂▁▁▁▁▂▁▁▂▁▁▂▂▂▂▂▁▂▃▄▂▂▃▃▆▄▆▃█▄█▆█▄
C/acc,██▇▇▇▇▆▂▁
C/forget_loss,█████████▇██▇█▇▇▇▇▇██▇▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
C/retain_loss,▂▁▁▁▁▁▁▁▁▂▁▁▂▂▂█▁▂▃▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁
D/acc,███▇▇▇▇▁▁
D/forget_loss,█▆▄▄▄▄▃▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▂▂▂▁▂▁▂▁▁▁
D/retain_loss,▁▁▁▁▁▁▁▁▁▁▁▄▂▂▁▇▁▁█▁▂▂▁▁▃▁▂▂▁▂▂▁▂▂▁▂▁▂▄▃


In [6]:
# super attack

# also do the predetermined folds? informationally separate is fine
# why does that work ?
from relearn.attacks import train_rtt

records = data[Datasets.WMDP]

folds = get_folds_shuffled(records, k)

for i in range(k):

    train_fold_inds = range(i + 1)
    eval_fold_inds = range(i + 1, k)

    train_dict = {fold_name(i): folds[i]["corpus"] for i in train_fold_inds}

    eval_dict = {fold_name(i): folds[i]["val"] for i in eval_fold_inds}

    model = train_rtt(
        model,
        train_dict,
        eval_dict,
        lr=5e-5,
        tokenizer=tokenizer,
        use_wandb=True,
        eval_at_start=True if i == 0 else False,
        n_epochs=2,
        max_batches=None,
        base_epoch=0,
    )

ImportError: cannot import name 'train_rtt' from 'relearn.attacks' (/mnt/align4_drive/tcqian/unlearn_order/src/relearn/attacks/__init__.py)